In [1]:
# ! pip install streamlit
# !pip install numba

In [3]:
# From: https://python.langchain.com/v0.1/docs/modules/memory/
import time
# from numba import cuda 
import gc
# import torch

from langchain_community.llms import LlamaCpp
# from langchain.callbacks.manager import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain_core.prompts import PromptTemplate

from langchain.chains import LLMChain

from langchain.memory import ConversationBufferMemory

from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [4]:
# first initialize the large language model
model_path = '..\models\llama-2-7b-chat.Q8_0.gguf'  # https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF

temperature = 0
top_p = 1
max_new_tokens = 500

# Define model parameters
time_1 = time.time()
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
            model_path = model_path,
            temperature = temperature,
            max_tokens = max_new_tokens,
            top_p = top_p,
            n_gpu_layers = -1,
            n_batch = 512, # Should be between 1 and n_ctx, consider the amount of RAM
            n_ctx = 4096,
            f16_kv = True,  # MUST set to True, otherwise you will run into problem after a couple of calls
            # callback_manager = callback_manager,
            verbose = True, # Verbose is required to pass to the callback manager
        )

print()
time_2 = time.time()
time_1_2 = time_2 - time_1
print(f'Elapsed time: {time_1_2:.2f} seconds')

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ..\models\llama-2-7b-chat.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32        


Elapsed time: 2.97 seconds


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | 
Model metadata: {'general.name': 'LLaMA v2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'llama.rope.dimension_count': '128', 'llama.embedding_length': '4096', 'llama.block_count': '32', 'llama.feed_forward_length': '11008', 'llama.attention.head_count': '32', 'tokenizer.ggml.eos_token_id': '2', 'general.file_type': '7', 'llama.attention.head_count_kv': '32', 'llama.attention.layer_norm_rms_epsilon': '0.000001', 'tokenizer.ggml.model': 'llama', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.unknown_token_id': '0'}
Using fallback chat format: llama-2


In [5]:
time_1 = time.time()

system_prompt = """You are a helpful, respectful and honest assistant. 
    Always answer as helpfully as possible.  
    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
    If you don't know the answer to a question, please don't share false information.
"""

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            system_prompt
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name = "chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key = "chat_history", return_messages= True)
conversation = LLMChain(
    llm = llm,
    prompt = prompt,
    verbose = True,
    memory = memory
)

print()
time_2 = time.time()
time_1_2 = time_2 - time_1
print(f'Elapsed time: {time_1_2:.2f} seconds')


Elapsed time: 0.00 seconds


In [6]:
time_1 = time.time()

user_prompt = """
Tell me a slightly funny story.
"""
answer_0 = conversation({"question": user_prompt})
print(answer_0['text'])

print()
time_2 = time.time()
time_1_2 = time_2 - time_1
print(f'Elapsed time: {time_1_2:.2f} seconds')

C:\Users\014206631\AppData\Local\miniconda3\envs\llamaChat\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful, respectful and honest assistant. 
    Always answer as helpfully as possible.  
    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
    If you don't know the answer to a question, please don't share false information.

Human: 
Tell me a slightly funny story.




llama_print_timings:        load time =    4071.52 ms
llama_print_timings:      sample time =      41.17 ms /   127 runs   (    0.32 ms per token,  3084.77 tokens per second)
llama_print_timings: prompt eval time =    4071.29 ms /    95 tokens (   42.86 ms per token,    23.33 tokens per second)
llama_print_timings:        eval time =   31035.99 ms /   126 runs   (  246.32 ms per token,     4.06 tokens per second)
llama_print_timings:       total time =   35563.31 ms /   221 tokens



> Finished chain.
Assistant:  Of course! Here is a slightly funny story for you:
Once upon a time, there was a chicken named Cluck who loved to play hide and seek with his friends. One day, while hiding behind a tree, he forgot to cover his beak and it got stuck in the branch. His friends found him and burst out laughing at the sight of him stuck there. Cluck was not amused and said, "What's so funny? I'm just trying to hide!"
I hope you enjoyed that story! Is there anything else I can help you with?

Elapsed time: 35.80 seconds


In [7]:
time_1 = time.time()

user_prompt = "What is the name of the chicken?"
answer_1 = conversation({"question": user_prompt})
print(answer_1['text'])

print()
time_2 = time.time()
time_1_2 = time_2 - time_1
print(f'Elapsed time: {time_1_2:.2f} seconds')

Llama.generate: prefix-match hit




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful, respectful and honest assistant. 
    Always answer as helpfully as possible.  
    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
    If you don't know the answer to a question, please don't share false information.

Human: 
Tell me a slightly funny story.

AI: Assistant:  Of course! Here is a slightly funny story for you:
Once upon a time, there was a chicken named Cluck who loved to play hide and seek with his friends. One day, while hiding behind a tree, he forgot to cover his beak and it got stuck in the branch. His friends found him and burst out laughing at the sight of him stuck there. Cluck was not amused and said, "What's so funny? I'm just trying to hide!"
I hope you enjoyed that story! Is there anything else I can help you with?
Human: What is the name of the chicken?



llama_print_timings:        load time =    4071.52 ms
llama_print_timings:      sample time =      18.53 ms /    57 runs   (    0.33 ms per token,  3076.92 tokens per second)
llama_print_timings: prompt eval time =    6456.20 ms /   142 tokens (   45.47 ms per token,    21.99 tokens per second)
llama_print_timings:        eval time =   12992.08 ms /    56 runs   (  232.00 ms per token,     4.31 tokens per second)
llama_print_timings:       total time =   19643.52 ms /   198 tokens



> Finished chain.
 
AI: Assistant: Oh, my apologies! The name of the chicken in the story is Cluck. I'm glad you liked it! If you have any other questions or would like to hear another slightly funny story, please let me know.

Elapsed time: 19.65 seconds


In [6]:
answer_1

{'question': 'What is the name of the chicken?',
 'chat_history': [HumanMessage(content='\nTell me a slightly funny story.\n'),
  AIMessage(content='Assistant:  Of course! Here is a slightly funny story for you:\nOnce upon a time, there was a chicken named Cluck who loved to play hide and seek with his friends. One day, while hiding behind a tree, he forgot to cover his beak and it got stuck in the branch. His friends found him and burst out laughing at the sight of him stuck there. Cluck was not amused and said, "What\'s so funny? I\'m just trying to hide!"\nI hope you enjoyed that story! Is there anything else I can help you with?'),
  HumanMessage(content='What is the name of the chicken?'),
  AIMessage(content=' \nAI: Assistant: Oh, good catch! The name of the chicken in the story is actually "Cluck Norris." Cluck is a common name for chickens, and Norris is a popular last name. I apologize for the confusion earlier. Is there anything else you would like to know?')],
 'text': ' \nA

In [8]:
print(answer_1['text'])

 
AI: Assistant: Oh, my apologies! The name of the chicken in the story is Cluck. I'm glad you liked it! If you have any other questions or would like to hear another slightly funny story, please let me know.


In [46]:
aa = answer_1['text'].strip()
aa

"AI: Assistant: Oh, my apologies! The name of the chicken in the story is Cluck. I'm glad you liked it! If you have any other questions or would like to hear another slightly funny story, please let me know."

In [52]:
answer_1['chat_history'][0]

HumanMessage(content='\nTell me a slightly funny story.\n')

In [57]:
print(answer_1['chat_history'][0])

content='\nTell me a slightly funny story.\n'


In [58]:
answer_1['chat_history'][0]

HumanMessage(content='\nTell me a slightly funny story.\n')

In [42]:
aa

" \nAI: Assistant: Oh, my apologies! The name of the chicken in the story is Cluck. I'm glad you liked it! If you have any other questions or would like to hear another slightly funny story, please let me know."

In [32]:
repr(aa)

'" \\nAI: Assistant: Oh, my apologies! The name of the chicken in the story is Cluck. I\'m glad you liked it! If you have any other questions or would like to hear another slightly funny story, please let me know."'

In [38]:
aa = print(answer_1['text'], flush = False, file = aa)

 
AI: Assistant: Oh, my apologies! The name of the chicken in the story is Cluck. I'm glad you liked it! If you have any other questions or would like to hear another slightly funny story, please let me know.


In [43]:
import sys
import io

old_stdout = sys.stdout # Memorize the default stdout stream
sys.stdout = buffer = io.StringIO()

print(answer_1['text'])

sys.stdout = old_stdout # Put the old stream back in place

aaa = buffer.getvalue() # Return a str containing the entire contents of the buffer.
print(aaa) # Why not to print it?
print(123)

 
AI: Assistant: Oh, my apologies! The name of the chicken in the story is Cluck. I'm glad you liked it! If you have any other questions or would like to hear another slightly funny story, please let me know.

123


In [45]:
aaa.strip()

"AI: Assistant: Oh, my apologies! The name of the chicken in the story is Cluck. I'm glad you liked it! If you have any other questions or would like to hear another slightly funny story, please let me know."

In [8]:
time_1 = time.time()

system_prompt = """You are a serious assistant that don't tell jokes.
"""

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            system_prompt
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name = "chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key = "chat_history", return_messages= True)
conversation = LLMChain(
    llm = llm,
    prompt = prompt,
    verbose = True,
    memory = memory
)

print()
time_2 = time.time()
time_1_2 = time_2 - time_1
print(f'Elapsed time: {time_1_2:.2f} seconds')


Elapsed time: 0.00 seconds


In [9]:
time_1 = time.time()

user_prompt = """
Tell me a slightly funny story.
"""
answer_0 = conversation({"question": user_prompt})
print(answer_0['text'])

print()
time_2 = time.time()
time_1_2 = time_2 - time_1
print(f'Elapsed time: {time_1_2:.2f} seconds')



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a serious assistant that don't tell jokes.

Human: 
Tell me a slightly funny story.



Llama.generate: prefix-match hit



> Finished chain.

System: I apologize, but I cannot fulfill that request as it goes against my programming rules rules rules rules to engage in humor or tell jokes. My purpose is to assist and provide helpful information, and I will continue to do so in a professional and respectful manner. Is there anything else I can help you with?

Elapsed time: 40.02 seconds


## streamlit version

In [ ]:
https://github.com/shashankdeshpande/langchain-chatbot/blob/master/pages/2_%E2%AD%90_context_aware_chatbot.py
https://langchain-chatbot.streamlit.app/context_aware_chatbot

In [ ]:
import streamlit as st 
# From: https://python.langchain.com/v0.1/docs/modules/memory/
import time
from langchain_community.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

from langchain.chains import LLMChain

from langchain.memory import ConversationBufferMemory

from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

class llama2Chat:
    def __init__(self):
        # first initialize the large language model
        self.model_path = '..\models\llama-2-7b-chat.Q8_0.gguf'  # https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF
        self.temperature = 0
        self.top_p = 1
        self.max_new_tokens = 500
        self.system_prompt = """You are a helpful, respectful and honest assistant. 
    Always answer as helpfully as possible.  
    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
    If you don't know the answer to a question, please don't share false information.
"""
    
    @st.cache_resource
    def setup_chain(_self):
        # Define model parameters
        callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
        llm = LlamaCpp(
                    model_path = _self.model_path,
                    temperature = _self.temperature,
                    max_tokens = _self.max_new_tokens,
                    top_p = _self.top_p,
                    n_gpu_layers = -1,
                    n_batch = 512, # Should be between 1 and n_ctx, consider the amount of RAM
                    n_ctx = 4096,
                    f16_kv = True,  # MUST set to True, otherwise you will run into problem after a couple of calls
                    callback_manager = callback_manager,
                    verbose = True, # Verbose is required to pass to the callback manager
                )
        
        prompt = ChatPromptTemplate(
            messages=[
                SystemMessagePromptTemplate.from_template(
                    _self.system_prompt
                ),
                # The `variable_name` here is what must align with memory
                MessagesPlaceholder(variable_name = "chat_history"),
                HumanMessagePromptTemplate.from_template("{question}")
            ]
        )
        memory = ConversationBufferMemory(memory_key = "chat_history", return_messages= True)
        _self.conversation = LLMChain(
            llm = llm,
            prompt = prompt,
            verbose = False,
            memory = memory
        )

    def answer_prompt(_self, user_prompt):
        _self.answer = _self.conversation({"question": user_prompt})   

In [ ]:
llamaChatObj = llama2Chat()

In [ ]:
llamaChatObj.setup_chain()

In [ ]:
user_prompt = """
Tell me a slightly funny story.
"""
llamaChatObj.answer_prompt(user_prompt)

In [ ]:
llamaChatObj.answer